# Transformations
This file will create a csv that is optimized for Tableau visualizations.


In [11]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

deq_df = pd.read_csv('csv_collection/deq_data_cleaned.csv')
counties_df = pd.read_csv('csv_collection/cleaned_counties.csv')

display(deq_df.head(3))
display(counties_df.head(3))

print(deq_df.info())
print(counties_df.info())


,permit_name,megawatts,total_acres,disturbance_zone_acres,mitigation_plan,city/county,brownfield_site,noi_received,application_received,pbr_authorization_letter,construction_commenced,commercial_operation_commenced
0,360 Solar Center Project,52.0,800,450,YES,Chesterfield County,No,7/19/22,4/29/24,7/15/24,NaN,NaN
1,Altavista Solar LLC (FKA Dragonfly),80.0,"1,200",0,YES,Campbell County,No,9/12/18,5/31/19,8/8/19,3/11/20,6/4/21
2,"Alton Post Office Solar, LLC",75.0,768,501,No,Halifax County,No,10/6/17,9/15/20,12/7/20,9/20/23,12/24/24


,city/county,biden_votes,biden_%,trump_votes,trump_%,other_party_votes,other_party_%,margin_votes,margin_%,total_votes,population,area,pop_density_sqmi,affiliation_2020,median_household_income,bachelors_or_over_%,age_over_50_%
0,Accomack County,7578,44.68,9172,54.07,212,1.25,-1594,-9.39,16962,33411.0,455.0,73.430769,red,57500.0,21.8,47.3
1,Albemarle County,42466,65.68,20804,32.18,1387,2.14,21662,33.50,64657,117313.0,723.0,162.258645,blue,102617.0,60.6,38.3
2,Alexandria City,66240,80.28,14544,17.63,1724,2.09,51696,62.65,82508,NaN,NaN,NaN,blue,113638.0,65.8,30.2


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79 entries, 0 to 78
Data columns (total 12 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   permit_name                     79 non-null     object 
 1   megawatts                       79 non-null     float64
 2   total_acres                     79 non-null     object 
 3   disturbance_zone_acres          73 non-null     object 
 4   mitigation_plan                 76 non-null     object 
 5   city/county                     79 non-null     object 
 6   brownfield_site                 79 non-null     object 
 7   noi_received                    79 non-null     object 
 8   application_received            79 non-null     object 
 9   pbr_authorization_letter        79 non-null     object 
 10  construction_commenced          45 non-null     object 
 11  commercial_operation_commenced  39 non-null     object 
dtypes: float64(1), object(11)
memory usage

In [12]:
# Fixing the acres column:

deq_df['total_acres'] = deq_df['total_acres'].apply(lambda x: float(x.replace(',', '')))

# Creating a square mile column
deq_df['total_project_mi'] = deq_df['total_acres'] * 0.0015625

print(deq_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79 entries, 0 to 78
Data columns (total 13 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   permit_name                     79 non-null     object 
 1   megawatts                       79 non-null     float64
 2   total_acres                     79 non-null     float64
 3   disturbance_zone_acres          73 non-null     object 
 4   mitigation_plan                 76 non-null     object 
 5   city/county                     79 non-null     object 
 6   brownfield_site                 79 non-null     object 
 7   noi_received                    79 non-null     object 
 8   application_received            79 non-null     object 
 9   pbr_authorization_letter        79 non-null     object 
 10  construction_commenced          45 non-null     object 
 11  commercial_operation_commenced  39 non-null     object 
 12  total_project_mi                79 non

Removing unnecessary columns:

In [13]:
deq_df = deq_df.drop(columns=['disturbance_zone_acres', 'brownfield_site', 'mitigation_plan', 'noi_received'])

In [14]:
counties_df = counties_df.drop(columns=['biden_votes', 'biden_%',
                                        'trump_votes', 'trump_%', 'other_party_votes', 'other_party_%', 'margin_votes'])

In [15]:
print(deq_df.info())
print(counties_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79 entries, 0 to 78
Data columns (total 9 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   permit_name                     79 non-null     object 
 1   megawatts                       79 non-null     float64
 2   total_acres                     79 non-null     float64
 3   city/county                     79 non-null     object 
 4   application_received            79 non-null     object 
 5   pbr_authorization_letter        79 non-null     object 
 6   construction_commenced          45 non-null     object 
 7   commercial_operation_commenced  39 non-null     object 
 8   total_project_mi                79 non-null     float64
dtypes: float64(3), object(6)
memory usage: 5.7+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 134 entries, 0 to 133
Data columns (total 10 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------

Creating a csv that groups by county, and also contains aggregates of the solar farms in that county:

In [16]:
# First, standardize the 'city/county' column name for merging
deq_df.rename(columns={'city/county': 'locality'}, inplace=True)
counties_df.rename(columns={'city/county': 'locality'}, inplace=True)

# Group projects by locality to calculate total MW and project count per locality
project_agg = deq_df.groupby('locality').agg({
    'megawatts': 'sum',
    'permit_name': 'count'
}).reset_index().rename(columns={
    'megawatts': 'total_megawatts',
    'permit_name': 'project_count'
})

# Merge aggregated project data with locality sociodemographics
merged_df = pd.merge(counties_df, project_agg, on='locality', how='left')

# Fill NaN for localities with 0 projects
merged_df[['total_megawatts', 'project_count']] = merged_df[['total_megawatts', 'project_count']].fillna(0)

# Aggregated columns for tabluea:
merged_df['megawatts_per_sqmi'] = merged_df['total_megawatts'] / merged_df['area']
merged_df['projects_per_100k_people'] = (merged_df['project_count'] / merged_df['population']) * 100000
merged_df['megwatts_per_1000_people'] = (merged_df['total_megawatts'] / merged_df['population']) * 1000



aggregated_localities_df = merged_df

In [17]:
# Only looking at counties that have solar projects:

aggregated_localities_df = aggregated_localities_df[aggregated_localities_df['project_count'] > 0]

aggregated_localities_df = aggregated_localities_df[aggregated_localities_df['locality'].str.contains('County')]

display(aggregated_localities_df.head(3))

,locality,margin_%,total_votes,population,area,pop_density_sqmi,affiliation_2020,median_household_income,bachelors_or_over_%,age_over_50_%,total_megawatts,project_count,megawatts_per_sqmi,projects_per_100k_people,megwatts_per_1000_people
0,Accomack County,-9.39,16962,33411.0,455.0,73.430769,red,57500.0,21.8,47.3,160.0,2.0,0.351648,5.986052,4.788842
1,Albemarle County,33.50,64657,117313.0,723.0,162.258645,blue,102617.0,60.6,38.3,138.0,1.0,0.190871,0.852420,1.176340
12,Botetourt County,-44.50,21120,34129.0,543.0,62.852670,red,80467.0,27.5,47.6,80.0,1.0,0.147330,2.930059,2.344048


In [18]:
aggregated_localities_df['state'] = 'Virginia'

display(aggregated_localities_df.head(3))

,locality,margin_%,total_votes,population,area,pop_density_sqmi,affiliation_2020,median_household_income,bachelors_or_over_%,age_over_50_%,total_megawatts,project_count,megawatts_per_sqmi,projects_per_100k_people,megwatts_per_1000_people,state
0,Accomack County,-9.39,16962,33411.0,455.0,73.430769,red,57500.0,21.8,47.3,160.0,2.0,0.351648,5.986052,4.788842,Virginia
1,Albemarle County,33.50,64657,117313.0,723.0,162.258645,blue,102617.0,60.6,38.3,138.0,1.0,0.190871,0.852420,1.176340,Virginia
12,Botetourt County,-44.50,21120,34129.0,543.0,62.852670,red,80467.0,27.5,47.6,80.0,1.0,0.147330,2.930059,2.344048,Virginia


In [19]:
# Saving as a csv:
aggregated_counties_df = aggregated_localities_df

aggregated_counties_df = aggregated_counties_df.rename(columns={'locality': 'county'})

aggregated_counties_df.to_csv('aggregated_counties.csv')